In [ ]:
import torch
from ultralytics import YOLO
import os
import numpy as np
import cv2

# YOLOv8-pose 모델 로드
model = YOLO('yolov8n-pose.pt')  # Pre-trained YOLOv8-pose 모델 사용

# 경로 설정
image_folder = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/aegis_pose_data/violence_B_30fps"  # 입력 이미지 경로
output_folder = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/filtered_keypoints"  # 결과 저장 경로
os.makedirs(output_folder, exist_ok=True)

# 키포인트 인덱스 (COCO 기준)
KEYPOINTS_INDEX = {
    "left_shoulder": 5, "left_elbow": 7, "left_wrist": 9,
    "right_shoulder": 6, "right_elbow": 8, "right_wrist": 10
}

# 바닥 키포인트 필터링 및 유효 키포인트 추출
def filter_keypoints(keypoints, confidences, image_height):
    """
    바닥 키포인트 제거 (하단 30%) 및 신뢰도 높은 키포인트만 필터링
    """
    filtered_keypoints = {}
    for name, idx in KEYPOINTS_INDEX.items():
        if idx < len(keypoints):
            x, y = keypoints[idx]
            conf = confidences[idx]

            # 조건: 신뢰도가 높고 바닥(하단 30%)에 위치하지 않은 키포인트만 저장
            if conf > 0.5 and y < image_height * 0.7:  # 하단 30% 필터링
                filtered_keypoints[name] = (int(x), int(y))
    return filtered_keypoints

# 이미지에서 키포인트 추출 및 필터링 (여러 사람 처리)
def extract_keypoints(image_path):
    image = cv2.imread(image_path)  # OpenCV로 이미지 불러오기
    results = model(image, verbose=False)  # YOLOv8-pose 실행

    people_keypoints = []  # 여러 사람의 키포인트 저장 리스트

    for result in results[0].keypoints:
        if result.xy is not None:
            keypoints = result.xy[0].cpu().numpy()  # 키포인트 좌표
            confidences = result.conf[0].cpu().numpy()  # 키포인트 신뢰도
            image_height = image.shape[0]

            # 유효 키포인트 필터링 적용
            filtered_keypoints = filter_keypoints(keypoints, confidences, image_height)
            if filtered_keypoints:  # 유효 키포인트가 존재하는 경우만 저장
                people_keypoints.append(filtered_keypoints)

    return people_keypoints

# 전체 이미지 처리
valid_keypoints = {}
for image_file in sorted(os.listdir(image_folder)):
    if image_file.lower().endswith(('.jpg', '.png')):
        image_path = os.path.join(image_folder, image_file)
        people_keypoints = extract_keypoints(image_path)
        if people_keypoints:  # 유효 키포인트가 존재하면 저장
            valid_keypoints[image_file] = people_keypoints
        print(f"{image_file}: {'Keypoints Extracted' if people_keypoints else 'No Valid Keypoints'}")

# 결과 저장
output_file = os.path.join(output_folder, "valid_keypoints_multiple.npy")
np.save(output_file, valid_keypoints)
print("Valid keypoints extraction completed and saved to 'valid_keypoints_multiple.npy'.")


### 키포인트 추출 이미지로 시각화

In [ ]:
import numpy as np
import os
import cv2

# 경로 설정
keypoints_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/filtered_keypoints/valid_keypoints_multiple.npy"  # 키포인트 파일 경로
image_folder = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/aegis_pose_data/violence_B_30fps"  # 원본 이미지 폴더
output_folder = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/keypoint"  # 결과 이미지 저장 폴더
os.makedirs(output_folder, exist_ok=True)

# 키포인트 데이터 불러오기
keypoints_data = np.load(keypoints_file, allow_pickle=True).item()

# 키포인트 시각화 함수
def visualize_keypoints(image_path, keypoints_list, output_path):
    image = cv2.imread(image_path)
    colors = [(0, 0, 255), (0, 255, 0), (255, 0, 0), (255, 255, 0)]  # 각 사람의 키포인트 색상

    for idx, keypoints in enumerate(keypoints_list):  # 여러 사람 순회
        color = colors[idx % len(colors)]  # 색상 선택 (순환)
        for name, (x, y) in keypoints.items():
            cv2.circle(image, (x, y), 5, color, -1)  # 키포인트 그리기
            cv2.putText(image, name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)  # 라벨 표시
    
    cv2.imwrite(output_path, image)  # 결과 이미지 저장

# 전체 이미지 처리 및 시각화
for image_file, keypoints_list in keypoints_data.items():
    image_path = os.path.join(image_folder, image_file)
    output_path = os.path.join(output_folder, image_file)
    if os.path.exists(image_path) and keypoints_list:  # 이미지가 존재하고 키포인트가 있으면
        visualize_keypoints(image_path, keypoints_list, output_path)
        print(f"Processed: {image_file}")
    else:
        print(f"Skipped: {image_file}")

print(f"Visualization complete. Images saved in {output_folder}")


### 좌표변화량 계산

In [ ]:
import numpy as np
import os

# 경로 설정
keypoints_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/filtered_keypoints/valid_keypoints_multiple.npy"
output_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/coordinate_changes.npy"

# 데이터 로드
keypoints_data = np.load(keypoints_file, allow_pickle=True).item()

# 좌표 변화량 계산 함수
def calculate_coordinate_changes(data):
    previous_coords = {}
    changes = {}

    for frame, people in sorted(data.items()):
        frame_changes = []
        for person in people:  # 여러 사람의 키포인트
            person_changes = []
            for key in ["left_shoulder", "left_elbow", "left_wrist", "right_shoulder", "right_elbow", "right_wrist"]:
                current_coords = person.get(key, (0, 0))
                previous_coords.setdefault(key, current_coords)
                delta = abs(current_coords[0] - previous_coords[key][0]) + abs(current_coords[1] - previous_coords[key][1])
                person_changes.append(delta)
                previous_coords[key] = current_coords
            frame_changes.append(person_changes)
        changes[frame] = frame_changes
    return changes

# 변화량 계산
coordinate_changes = calculate_coordinate_changes(keypoints_data)

# 결과 저장
np.save(output_file, coordinate_changes)
print(f"Coordinate changes saved to: {output_file}")


### 좌표 변화량 시각화

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 경로 설정
coordinate_changes_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/coordinate_changes.npy"

# 변화량 데이터 불러오기
coordinate_changes = np.load(coordinate_changes_file, allow_pickle=True).item()

# 프레임 순서대로 정렬된 데이터 준비
frames = sorted(coordinate_changes.keys())  # 프레임 이름 정렬
keypoints_names = ["left_shoulder", "left_elbow", "left_wrist", "right_shoulder", "right_elbow", "right_wrist"]

# 변화량 시각화
def plot_average_coordinate_changes(coordinate_changes):
    plt.figure(figsize=(12, 8))

    # 각 키포인트별로 변화량 추출 및 평균 시각화
    for i, keypoint_name in enumerate(keypoints_names):
        average_changes = []  # 프레임당 평균 변화량 저장

        for frame in frames:
            people_changes = coordinate_changes[frame]  # 각 프레임의 모든 사람 변화량
            keypoint_sum = 0
            keypoint_count = 0

            for person_changes in people_changes:
                if len(person_changes) == 6:  # 어깨, 팔꿈치, 손목 변화량
                    keypoint_sum += person_changes[i]
                    keypoint_count += 1

            # 평균 변화량 계산
            avg_change = keypoint_sum / keypoint_count if keypoint_count > 0 else 0
            average_changes.append(avg_change)

        # 그래프 그리기
        plt.plot(range(len(frames)), average_changes, label=f"{keypoint_name}_avg")

    plt.title("Average Keypoint Coordinate Changes Over Frames")
    plt.xlabel("Frame Index")
    plt.ylabel("Average Coordinate Changes")
    plt.legend()
    plt.grid(True)
    plt.show()

# 변화량 시각화 실행
plot_average_coordinate_changes(coordinate_changes)


### 키포인트 보간

In [ ]:
import torch
from ultralytics import YOLO
import os
import numpy as np
import cv2

# YOLOv8-pose 모델 로드
model = YOLO('yolov8n-pose.pt')

# 경로 설정
image_folder = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/aegis_pose_data/violence_B_30fps"
output_folder = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/filtered_keypoints_improved"
os.makedirs(output_folder, exist_ok=True)

# 키포인트 인덱스 (COCO 포맷 기준)
KEYPOINTS_INDEX = {
    "left_shoulder": 5, "left_elbow": 7, "left_wrist": 9,
    "right_shoulder": 6, "right_elbow": 8, "right_wrist": 10
}

# 바닥 필터링 및 신뢰도 필터링
def filter_keypoints(keypoints, confidences, image_height):
    filtered_keypoints = {}
    for name, idx in KEYPOINTS_INDEX.items():
        if idx < len(keypoints):
            x, y = keypoints[idx]
            conf = confidences[idx]

            # 신뢰도 강화 (0.7) 및 바닥 필터링
            if conf > 0.7 and y < image_height * 0.7:
                filtered_keypoints[name] = (int(x), int(y))
    return filtered_keypoints

# 키포인트 보간 함수
def interpolate_keypoints(prev_keypoints, next_keypoints):
    interpolated = {}
    for key in KEYPOINTS_INDEX.keys():
        if prev_keypoints.get(key) and next_keypoints.get(key):
            x = (prev_keypoints[key][0] + next_keypoints[key][0]) // 2
            y = (prev_keypoints[key][1] + next_keypoints[key][1]) // 2
            interpolated[key] = (x, y)
    return interpolated

# 이미지 처리 및 키포인트 추출
valid_keypoints = {}
prev_keypoints = None

for image_file in sorted(os.listdir(image_folder)):
    if image_file.lower().endswith(('.jpg', '.png')):
        image_path = os.path.join(image_folder, image_file)
        image = cv2.imread(image_path)
        results = model(image, verbose=False)

        current_keypoints = None
        for result in results[0].keypoints:
            if result.xy is not None:
                keypoints = result.xy[0].cpu().numpy()
                confidences = result.conf[0].cpu().numpy()
                current_keypoints = filter_keypoints(keypoints, confidences, image.shape[0])

        # 보간 처리
        if not current_keypoints and prev_keypoints:
            current_keypoints = interpolate_keypoints(prev_keypoints, prev_keypoints)
        if current_keypoints:
            valid_keypoints[image_file] = current_keypoints
            prev_keypoints = current_keypoints  # 이전 키포인트 업데이트

        print(f"{image_file}: {'Keypoints Extracted' if current_keypoints else 'No Keypoints'}")

# 결과 저장
output_file = os.path.join(output_folder, "valid_keypoints_improved.npy")
np.save(output_file, valid_keypoints)
print("Improved keypoints extraction completed and saved.")


### 시각화

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 경로 설정
keypoints_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/filtered_keypoints_improved/valid_keypoints_improved.npy"

# 데이터 로드
keypoints_data = np.load(keypoints_file, allow_pickle=True).item()

# 키포인트 변화량 계산 함수
def calculate_coordinate_changes(data):
    previous_coords = {}
    changes = {key: [] for key in ["left_shoulder", "left_elbow", "left_wrist", "right_shoulder", "right_elbow", "right_wrist"]}
    frame_indices = []

    for frame_index, (frame, keypoints) in enumerate(sorted(data.items())):
        frame_indices.append(frame_index)

        for key in changes.keys():
            current_coords = keypoints.get(key, (0, 0))
            if key in previous_coords:
                delta = abs(current_coords[0] - previous_coords[key][0]) + abs(current_coords[1] - previous_coords[key][1])
            else:
                delta = 0  # 첫 프레임은 변화량 0
            changes[key].append(delta)
            previous_coords[key] = current_coords

    return frame_indices, changes

# 변화량 계산
frame_indices, coordinate_changes = calculate_coordinate_changes(keypoints_data)

# 변화량 시각화
def plot_coordinate_changes(frame_indices, changes):
    plt.figure(figsize=(12, 8))

    for key, deltas in changes.items():
        plt.plot(frame_indices, deltas, label=f"{key}_change")

    plt.title("Keypoint Coordinate Changes Over Frames")
    plt.xlabel("Frame Index")
    plt.ylabel("Coordinate Changes")
    plt.legend()
    plt.grid(True)
    plt.show()

# 그래프 출력
plot_coordinate_changes(frame_indices, coordinate_changes)


### 오른쪽 온리 키포인트

In [ ]:
import torch
from ultralytics import YOLO
import os
import numpy as np
import cv2

# YOLOv8-pose 모델 로드
model = YOLO('yolov8n-pose.pt')

# 경로 설정
image_folder = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/aegis_pose_data/violence_B_30fps"
output_folder = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/filtered_keypoints_right_only"
os.makedirs(output_folder, exist_ok=True)

# 키포인트 인덱스 (COCO 기준)
KEYPOINTS_INDEX = {
    "left_shoulder": 5, "left_elbow": 7, "left_wrist": 9,
    "right_shoulder": 6, "right_elbow": 8, "right_wrist": 10
}

# 바닥 키포인트 필터링 및 오른쪽 사람 선택
def filter_keypoints_and_select_rightmost(results, image_height):
    rightmost_keypoints = {}
    max_x = -1  # x좌표 최대값 초기화

    for result in results.keypoints:  # 모든 사람의 키포인트 순회
        if result.xy is not None:
            keypoints = result.xy[0].cpu().numpy()
            confidences = result.conf[0].cpu().numpy()
            filtered_keypoints = {}

            # 키포인트 필터링 (신뢰도 & 바닥 제거)
            for name, idx in KEYPOINTS_INDEX.items():
                x, y = keypoints[idx]
                conf = confidences[idx]
                if conf > 0.7 and y < image_height * 0.7:  # 신뢰도와 바닥 필터링
                    filtered_keypoints[name] = (int(x), int(y))

            # 오른쪽 사람 선택 (x좌표가 가장 큰 사람)
            if filtered_keypoints:
                avg_x = sum(x for x, _ in filtered_keypoints.values()) / len(filtered_keypoints)
                if avg_x > max_x:
                    max_x = avg_x
                    rightmost_keypoints = filtered_keypoints

    return rightmost_keypoints

# 이미지 처리 및 오른쪽 사람 키포인트 추출
valid_keypoints = {}

for image_file in sorted(os.listdir(image_folder)):
    if image_file.lower().endswith(('.jpg', '.png')):
        image_path = os.path.join(image_folder, image_file)
        image = cv2.imread(image_path)
        results = model(image, verbose=False)

        # 오른쪽 사람 키포인트 추출
        keypoints = filter_keypoints_and_select_rightmost(results[0], image.shape[0])
        if keypoints:
            valid_keypoints[image_file] = keypoints
            print(f"{image_file}: Keypoints Extracted")
        else:
            print(f"{image_file}: No Keypoints")

# 결과 저장
output_file = os.path.join(output_folder, "valid_keypoints_right_only.npy")
np.save(output_file, valid_keypoints)
print("Rightmost keypoints extraction completed and saved.")


### 오른쪽 사람 키포인트 확인

In [ ]:
import numpy as np
import os
import cv2

# 경로 설정
keypoints_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/filtered_keypoints_right_only/valid_keypoints_right_only.npy"
image_folder = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/aegis_pose_data/violence_B_30fps"
output_folder = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/keypoint"
os.makedirs(output_folder, exist_ok=True)

# 키포인트 시각화 함수
def visualize_keypoints(image_path, keypoints, output_path):
    image = cv2.imread(image_path)
    for name, (x, y) in keypoints.items():  # 키포인트 순회
        cv2.circle(image, (x, y), 5, (0, 0, 255), -1)  # 빨간색 점 그리기
        cv2.putText(image, name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)  # 키포인트 이름 표시
    cv2.imwrite(output_path, image)  # 시각화된 이미지 저장

# 키포인트 파일 불러오기
keypoints_data = np.load(keypoints_file, allow_pickle=True).item()

# 이미지 시각화 실행
for image_file, keypoints in keypoints_data.items():
    image_path = os.path.join(image_folder, image_file)
    output_path = os.path.join(output_folder, image_file)
    if os.path.exists(image_path) and keypoints:
        visualize_keypoints(image_path, keypoints, output_path)
        print(f"Visualized: {image_file}")
    else:
        print(f"Skipped: {image_file}")

print(f"Keypoint visualization complete. Images saved in {output_folder}")


### 키포인트 시각화

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 경로 설정
keypoints_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/filtered_keypoints_right_only/valid_keypoints_right_only.npy"

# 키포인트 데이터 불러오기
keypoints_data = np.load(keypoints_file, allow_pickle=True).item()

# 키포인트 변화량 계산 함수 (존재하던 키포인트만)
def calculate_filtered_changes(data):
    previous_coords = {}
    changes = {key: [] for key in ["left_shoulder", "left_elbow", "left_wrist", "right_shoulder", "right_elbow", "right_wrist"]}
    frame_indices = []

    for frame_index, (frame, keypoints) in enumerate(sorted(data.items())):
        frame_indices.append(frame_index)
        
        for key in changes.keys():
            current_coords = keypoints.get(key, None)  # 현재 프레임 키포인트
            if key in previous_coords and current_coords:  
                # 존재하던 키포인트의 변화량만 계산
                delta = abs(current_coords[0] - previous_coords[key][0]) + abs(current_coords[1] - previous_coords[key][1])
                changes[key].append(delta)
            else:
                # 키포인트가 새로 생성되었거나 누락된 경우 0으로 처리
                changes[key].append(0)
            
            # 이전 키포인트 업데이트 (현재 좌표가 있는 경우만)
            if current_coords:
                previous_coords[key] = current_coords

    return frame_indices, changes

# 변화량 계산
frame_indices, filtered_changes = calculate_filtered_changes(keypoints_data)

# 변화량 시각화 함수
def plot_filtered_changes(frame_indices, changes):
    plt.figure(figsize=(12, 8))

    for key, deltas in changes.items():
        plt.plot(frame_indices, deltas, label=f"{key}_change")

    plt.title("Filtered Keypoint Coordinate Changes Over Frames")
    plt.xlabel("Frame Index")
    plt.ylabel("Coordinate Changes")
    plt.legend()
    plt.grid(True)
    plt.show()

# 그래프 출력
plot_filtered_changes(frame_indices, filtered_changes)


### 오른쪽 손목팔꿈치 변화량 그래프

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 경로 설정
keypoints_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/filtered_keypoints_right_only/valid_keypoints_right_only.npy"

# 키포인트 데이터 불러오기
keypoints_data = np.load(keypoints_file, allow_pickle=True).item()

# 오른쪽 손목과 팔꿈치 변화량 계산 함수
def calculate_filtered_changes(data):
    previous_coords = {}
    changes = {"right_elbow": [], "right_wrist": []}  # 오른쪽 팔꿈치와 손목만 고려
    frame_indices = []

    for frame_index, (frame, keypoints) in enumerate(sorted(data.items())):
        frame_indices.append(frame_index)

        for key in changes.keys():  # 오른쪽 팔꿈치와 손목만 순회
            current_coords = keypoints.get(key, None)  # 현재 프레임 키포인트
            if key in previous_coords and current_coords:  
                # 존재하던 키포인트의 변화량만 계산
                delta = abs(current_coords[0] - previous_coords[key][0]) + abs(current_coords[1] - previous_coords[key][1])
                changes[key].append(delta)
            else:
                # 키포인트가 새로 생성되었거나 누락된 경우 0으로 처리
                changes[key].append(0)
            
            # 이전 키포인트 업데이트 (현재 좌표가 있는 경우만)
            if current_coords:
                previous_coords[key] = current_coords

    return frame_indices, changes

# 변화량 계산
frame_indices, filtered_changes = calculate_filtered_changes(keypoints_data)

# 변화량 시각화 함수
def plot_filtered_changes(frame_indices, changes):
    plt.figure(figsize=(12, 8))

    # 오른쪽 팔꿈치와 손목만 시각화
    for key, deltas in changes.items():
        plt.plot(frame_indices, deltas, label=f"{key}_change", linewidth=2)

    plt.title("Right Elbow and Wrist Coordinate Changes Over Frames")
    plt.xlabel("Frame Index")
    plt.ylabel("Coordinate Changes")
    plt.legend()
    plt.grid(True)
    plt.show()

# 그래프 출력
plot_filtered_changes(frame_indices, filtered_changes)


### 왼쪽 생성

In [ ]:
import numpy as np

# 데이터 로드
keypoints_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/filtered_keypoints_right_only/valid_keypoints_right_only.npy"
keypoints_data = np.load(keypoints_file, allow_pickle=True).item()

# 대체 왼쪽 키포인트 생성 함수
def generate_left_keypoints(data, image_width):
    new_data = {}
    for frame, keypoints in data.items():
        new_keypoints = keypoints.copy()

        # 왼쪽 손목과 팔꿈치 대체 생성 (대칭 처리)
        if "right_wrist" in keypoints:
            x, y = keypoints["right_wrist"]
            new_keypoints["left_wrist"] = (image_width - x, y)

        if "right_elbow" in keypoints:
            x, y = keypoints["right_elbow"]
            new_keypoints["left_elbow"] = (image_width - x, y)

        new_data[frame] = new_keypoints
    return new_data

# 이미지 폭 (가정: 1920 픽셀)
IMAGE_WIDTH = 1920  

# 왼쪽 키포인트 보완
new_keypoints_data = generate_left_keypoints(keypoints_data, IMAGE_WIDTH)

# 결과 저장
output_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/filtered_keypoints_with_left.npy"
np.save(output_file, new_keypoints_data)
print(f"Left keypoints generated and saved to {output_file}")


# 왼쪽 오른쪽 시각화

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 경로 설정
keypoints_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/filtered_keypoints_with_left.npy"

# 키포인트 데이터 불러오기
keypoints_data = np.load(keypoints_file, allow_pickle=True).item()

# 키포인트 변화량 계산 함수
def calculate_changes(data):
    previous_coords = {}
    changes = {"left_elbow": [], "left_wrist": [], "right_elbow": [], "right_wrist": []}
    frame_indices = []

    for frame_index, (frame, keypoints) in enumerate(sorted(data.items())):
        frame_indices.append(frame_index)

        for key in changes.keys():
            current_coords = keypoints.get(key, None)  # 현재 프레임 키포인트
            if key in previous_coords and current_coords:
                # 존재하던 키포인트의 변화량만 계산
                delta = abs(current_coords[0] - previous_coords[key][0]) + abs(current_coords[1] - previous_coords[key][1])
                changes[key].append(delta)
            else:
                # 키포인트가 새로 생성되었거나 누락된 경우 0으로 처리
                changes[key].append(0)

            # 이전 키포인트 업데이트 (현재 좌표가 있는 경우만)
            if current_coords:
                previous_coords[key] = current_coords

    return frame_indices, changes

# 변화량 계산
frame_indices, changes = calculate_changes(keypoints_data)

# 변화량 시각화 함수
def plot_changes(frame_indices, changes):
    plt.figure(figsize=(12, 8))

    # 왼쪽과 오른쪽 키포인트의 변화량 시각화
    for key, deltas in changes.items():
        plt.plot(frame_indices, deltas, label=f"{key}_change", linewidth=2)

    plt.title("Left and Right Arm Keypoint Coordinate Changes Over Frames")
    plt.xlabel("Frame Index")
    plt.ylabel("Coordinate Changes")
    plt.legend()
    plt.grid(True)
    plt.show()

# 그래프 출력
plot_changes(frame_indices, changes)


### 왼쪽오른쪽 따로 그래프

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 경로 설정
keypoints_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/filtered_keypoints_with_left_fixed.npy"

# 키포인트 데이터 불러오기
keypoints_data = np.load(keypoints_file, allow_pickle=True).item()

# 키포인트 변화량 계산 함수
def calculate_changes(data):
    previous_coords = {}
    changes = {"left_elbow": [], "left_wrist": [], "right_elbow": [], "right_wrist": []}
    frame_indices = []

    for frame_index, (frame, keypoints) in enumerate(sorted(data.items())):
        frame_indices.append(frame_index)

        for key in changes.keys():
            current_coords = keypoints.get(key, None)  # 현재 프레임 키포인트
            if key in previous_coords and current_coords:
                # 존재하던 키포인트의 변화량만 계산
                delta = abs(current_coords[0] - previous_coords[key][0]) + abs(current_coords[1] - previous_coords[key][1])
                changes[key].append(delta)
            else:
                # 키포인트가 새로 생성되었거나 누락된 경우 0으로 처리
                changes[key].append(0)

            # 이전 키포인트 업데이트 (현재 좌표가 있는 경우만)
            if current_coords:
                previous_coords[key] = current_coords

    return frame_indices, changes

# 변화량 계산
frame_indices, changes = calculate_changes(keypoints_data)

# 왼쪽 키포인트 변화량 시각화 함수
def plot_left_changes(frame_indices, changes):
    plt.figure(figsize=(12, 6))
    plt.plot(frame_indices, changes["left_elbow"], label="left_elbow_change", linewidth=2, color='orange')
    plt.plot(frame_indices, changes["left_wrist"], label="left_wrist_change", linewidth=2, color='blue')
    plt.title("Left Arm Keypoint Coordinate Changes Over Frames")
    plt.xlabel("Frame Index")
    plt.ylabel("Coordinate Changes")
    plt.legend()
    plt.grid(True)
    plt.show()

# 오른쪽 키포인트 변화량 시각화 함수
def plot_right_changes(frame_indices, changes):
    plt.figure(figsize=(12, 6))
    plt.plot(frame_indices, changes["right_elbow"], label="right_elbow_change", linewidth=2, color='red')
    plt.plot(frame_indices, changes["right_wrist"], label="right_wrist_change", linewidth=2, color='green')
    plt.title("Right Arm Keypoint Coordinate Changes Over Frames")
    plt.xlabel("Frame Index")
    plt.ylabel("Coordinate Changes")
    plt.legend()
    plt.grid(True)
    plt.show()

# 그래프 출력
plot_left_changes(frame_indices, changes)  # 왼쪽 키포인트 그래프
plot_right_changes(frame_indices, changes)  # 오른쪽 키포인트 그래프


### 왼쪽을 픽스????

In [ ]:
import numpy as np

# 경로 설정
keypoints_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/filtered_keypoints_right_only/valid_keypoints_right_only.npy"
output_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/filtered_keypoints_with_left_fixed.npy"

# 대칭 왼쪽 키포인트 생성 함수
def generate_left_keypoints(data, image_width):
    new_data = {}
    for frame, keypoints in data.items():
        new_keypoints = keypoints.copy()

        # 오른쪽 손목과 팔꿈치를 기반으로 왼쪽 키포인트 생성
        if "right_wrist" in keypoints:
            x, y = keypoints["right_wrist"]
            new_keypoints["left_wrist"] = (image_width - x, y)  # x좌표만 반전

        if "right_elbow" in keypoints:
            x, y = keypoints["right_elbow"]
            new_keypoints["left_elbow"] = (image_width - x, y)  # x좌표만 반전

        new_data[frame] = new_keypoints
    return new_data

# 이미지 폭 설정 (가정: 1920 픽셀)
IMAGE_WIDTH = 1920

# 대칭 키포인트 생성
keypoints_data = np.load(keypoints_file, allow_pickle=True).item()
fixed_keypoints_data = generate_left_keypoints(keypoints_data, IMAGE_WIDTH)

# 결과 저장
np.save(output_file, fixed_keypoints_data)
print(f"Fixed left keypoints generated and saved to {output_file}")


### 변화량 다시

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 경로 설정
keypoints_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/filtered_keypoints_with_left_fixed.npy"

# 키포인트 데이터 불러오기
keypoints_data = np.load(keypoints_file, allow_pickle=True).item()

# 키포인트 변화량 계산 함수
def calculate_changes_with_initialization(data):
    previous_coords = {}
    changes = {"left_elbow": [], "left_wrist": [], "right_elbow": [], "right_wrist": []}
    frame_indices = []

    for frame_index, (frame, keypoints) in enumerate(sorted(data.items())):
        frame_indices.append(frame_index)

        for key in changes.keys():
            current_coords = keypoints.get(key, None)  # 현재 프레임 키포인트
            
            # 변화량 계산: 이전 좌표가 존재할 때만 계산
            if key in previous_coords and current_coords:
                delta = abs(current_coords[0] - previous_coords[key][0]) + abs(current_coords[1] - previous_coords[key][1])
                changes[key].append(delta)
            else:
                # 새로 등장하거나 누락된 경우 변화량 0
                changes[key].append(0)

            # 이전 키포인트 업데이트 (현재 좌표가 있는 경우만)
            if current_coords:
                previous_coords[key] = current_coords

    return frame_indices, changes

# 변화량 계산
frame_indices, changes = calculate_changes_with_initialization(keypoints_data)

# 왼쪽 키포인트 변화량 시각화
def plot_left_changes(frame_indices, changes):
    plt.figure(figsize=(12, 6))
    plt.plot(frame_indices, changes["left_elbow"], label="left_elbow_change", linewidth=2, color='orange')
    plt.plot(frame_indices, changes["left_wrist"], label="left_wrist_change", linewidth=2, color='blue')
    plt.title("Left Arm Keypoint Coordinate Changes Over Frames (Fixed)")
    plt.xlabel("Frame Index")
    plt.ylabel("Coordinate Changes")
    plt.legend()
    plt.grid(True)
    plt.show()

# 오른쪽 키포인트 변화량 시각화
def plot_right_changes(frame_indices, changes):
    plt.figure(figsize=(12, 6))
    plt.plot(frame_indices, changes["right_elbow"], label="right_elbow_change", linewidth=2, color='red')
    plt.plot(frame_indices, changes["right_wrist"], label="right_wrist_change", linewidth=2, color='green')
    plt.title("Right Arm Keypoint Coordinate Changes Over Frames")
    plt.xlabel("Frame Index")
    plt.ylabel("Coordinate Changes")
    plt.legend()
    plt.grid(True)
    plt.show()

# 그래프 출력
plot_left_changes(frame_indices, changes)  # 왼쪽 키포인트 그래프
plot_right_changes(frame_indices, changes)  # 오른쪽 키포인트 그래프


### 변화량 데이터 생성????

In [ ]:
import numpy as np

# 경로 설정
keypoints_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/filtered_keypoints_with_left_fixed.npy"
output_data_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/coordinate_changes_labeled.npy"

# 키포인트 인덱스 설정
KEYPOINTS = ["left_elbow", "left_wrist", "right_elbow", "right_wrist"]
SEQUENCE_LENGTH = 10  # 시퀀스 길이
THRESHOLD_FACTOR = 10  # 변화량이 10배 이상일 때 폭력으로 감지

# 변화량 계산 함수
def calculate_coordinate_changes(data):
    previous_coords = {}
    coordinate_changes = []
    labels = []

    for frame, keypoints in sorted(data.items()):
        frame_changes = []
        max_change = 0

        for key in KEYPOINTS:
            current_coords = keypoints.get(key, None)
            if key in previous_coords and current_coords:
                delta = abs(current_coords[0] - previous_coords[key][0]) + abs(current_coords[1] - previous_coords[key][1])
                max_change = max(max_change, delta)
                frame_changes.append(delta)
            else:
                frame_changes.append(0)  # 누락된 키포인트는 0으로 처리
            if current_coords:
                previous_coords[key] = current_coords

        coordinate_changes.append(frame_changes)
        labels.append(1 if max_change >= THRESHOLD_FACTOR else 0)

    return np.array(coordinate_changes), np.array(labels)

# 키포인트 데이터 로드 및 변화량 계산
keypoints_data = np.load(keypoints_file, allow_pickle=True).item()
coordinate_changes, labels = calculate_coordinate_changes(keypoints_data)

# 시퀀스 데이터 준비
X, y = [], []
for i in range(len(coordinate_changes) - SEQUENCE_LENGTH):
    X.append(coordinate_changes[i:i+SEQUENCE_LENGTH])
    y.append(max(labels[i:i+SEQUENCE_LENGTH]))  # 시퀀스 내 라벨 중 최대값 사용

X = np.array(X)
y = np.array(y)

# 데이터 저장
np.save(output_data_file, {"X": X, "y": y})
print(f"Coordinate changes and labels saved to {output_data_file}")


### 저장된 데이터 확인

In [ ]:
import numpy as np

# 저장된 파일 경로
output_data_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/coordinate_changes_labeled.npy"

# 데이터 불러오기
data = np.load(output_data_file, allow_pickle=True).item()
X, y = data["X"], data["y"]

# 데이터 형태 확인
print("Shape of X (input sequences):", X.shape)  # 입력 시퀀스의 형태
print("Shape of y (labels):", y.shape)  # 라벨의 형태

# 일부 데이터 내용 확인
print("\nFirst input sequence (X):")
print(X[0])  # 첫 번째 시퀀스 확인

print("\nCorresponding label (y):")
print(y[0])  # 첫 번째 시퀀스의 라벨 확인

# 통계 정보 확인
print("\nStatistics of Input Sequences (X):")
print("Max value:", X.max())
print("Min value:", X.min())
print("Mean value:", X.mean())

#
sequence_index = 0  # 첫 번째 시퀀스 확인
print(f"Frame-by-frame changes for Sequence {sequence_index}:")
print(X[sequence_index])  # 시퀀스 내 각 프레임 변화량



In [39]:
### 변화량데이터생성및 라벨링

In [ ]:
import numpy as np

# 경로 설정
keypoints_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/filtered_keypoints_with_left_fixed.npy"
output_data_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/labeled_lstm_data_with_ratio.npy"

# 하이퍼파라미터 설정
SEQUENCE_LENGTH = 10  # LSTM 입력 시퀀스 길이
THRESHOLD_FACTOR = 10  # 변화량 임계값
THRESHOLD_RATIO = 0.3  # 시퀀스 내 30% 이상 프레임이 임계값을 넘으면 폭력으로 인식

# 데이터 준비 함수
def prepare_data_with_ratio(data):
    previous_coords = {}
    sequence = []
    labels = []

    # 프레임별 변화량 계산
    for frame, keypoints in sorted(data.items()):
        frame_changes = []
        for key in ["right_wrist", "right_elbow", "left_wrist", "left_elbow"]:
            current_coords = keypoints.get(key, None)
            if key in previous_coords and current_coords:
                # 변화량 계산
                delta = abs(current_coords[0] - previous_coords[key][0]) + abs(current_coords[1] - previous_coords[key][1])
                frame_changes.append(delta)
            else:
                frame_changes.append(0)  # 키포인트가 없으면 변화량 0으로 설정
            if current_coords:
                previous_coords[key] = current_coords  # 이전 프레임 업데이트
        sequence.append(frame_changes)

    # 시퀀스 생성 및 라벨링
    X, y = [], []
    for i in range(len(sequence) - SEQUENCE_LENGTH):
        sub_sequence = sequence[i:i+SEQUENCE_LENGTH]  # 시퀀스 길이만큼 추출
        high_change_frames = sum(1 for frame in sub_sequence if max(frame) >= THRESHOLD_FACTOR)
        label = 1 if high_change_frames / SEQUENCE_LENGTH >= THRESHOLD_RATIO else 0
        X.append(sub_sequence)
        y.append(label)

    return np.array(X), np.array(y)

# 키포인트 데이터 불러오기
print("Loading keypoints data...")
keypoints_data = np.load(keypoints_file, allow_pickle=True).item()

# 변화량 데이터 준비
print("Calculating coordinate changes and labeling data...")
X, y = prepare_data_with_ratio(keypoints_data)

# 결과 저장
print(f"Saving labeled data to {output_data_file}...")
np.save(output_data_file, {"X": X, "y": y})

# 데이터 확인
print("Data preparation complete.")
print(f"Shape of X (input sequences): {X.shape}")
print(f"Shape of y (labels): {y.shape}")
print(f"Example sequence:\n{X[0]}")
print(f"Corresponding label: {y[0]}")


In [41]:
## 모든시퀀스 변화량

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 경로 설정
data_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/labeled_lstm_data_with_ratio.npy"

# 데이터 로드
data = np.load(data_file, allow_pickle=True).item()
X, y = data["X"], data["y"]

# 데이터 정보 출력
print("Shape of X (input sequences):", X.shape)  # (시퀀스 수, 시퀀스 길이, 키포인트 수)
print("Shape of y (labels):", y.shape)          # (시퀀스 수,)
print(f"Number of Violent (Label=1): {np.sum(y)}")
print(f"Number of Non-Violent (Label=0): {len(y) - np.sum(y)}")

# 시퀀스 평균 변화량 계산 (시퀀스 단위로 압축)
average_changes = X.mean(axis=1)  # 각 시퀀스에 대해 키포인트별 평균 변화량

# 전체 시퀀스 평균 변화량 시각화
def plot_all_sequences(average_changes, labels):
    plt.figure(figsize=(12, 8))
    for i in range(len(average_changes)):
        color = 'red' if labels[i] == 1 else 'blue'  # 폭력(1)은 빨간색, 비폭력(0)은 파란색
        plt.plot(range(len(average_changes[i])), average_changes[i], color=color, alpha=0.5)

    plt.title("Average Keypoint Changes for All Sequences")
    plt.xlabel("Frame Index in Sequence")
    plt.ylabel("Coordinate Changes")
    plt.grid(True)
    plt.show()

# 전체 시퀀스 시각화
plot_all_sequences(average_changes, y)


In [43]:
### 시퀀스별 평균 변화량 시각화

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 경로 설정
data_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/labeled_lstm_data_with_ratio.npy"

# 데이터 로드
data = np.load(data_file, allow_pickle=True).item()
X, y = data["X"], data["y"]

# 데이터 정보 출력
print("Shape of X (input sequences):", X.shape)  # (시퀀스 수, 시퀀스 길이, 키포인트 수)
print("Shape of y (labels):", y.shape)          # (시퀀스 수,)
print(f"Number of Sequences: {len(X)}")
print(f"Number of Violent (Label=1): {np.sum(y)}")
print(f"Number of Non-Violent (Label=0): {len(y) - np.sum(y)}")

# 시퀀스별 평균 변화량 계산
average_changes = X.mean(axis=(1, 2))  # 시퀀스 단위로 평균 변화량 계산

# 시퀀스별 평균 변화량 시각화
def plot_sequence_changes(average_changes, labels):
    plt.figure(figsize=(12, 6))

    # 시퀀스별 평균 변화량 그리기
    for i, (change, label) in enumerate(zip(average_changes, labels)):
        color = 'red' if label == 1 else 'blue'  # 폭력(1): 빨간색, 비폭력(0): 파란색
        plt.scatter(i, change, color=color, s=10)  # 각 시퀀스의 평균 변화량

    plt.title("Average Coordinate Changes for Each Sequence")
    plt.xlabel("Sequence Index")
    plt.ylabel("Average Coordinate Changes")
    plt.grid(True)
    plt.legend(["Violent (Label=1)", "Non-Violent (Label=0)"], loc="upper right")
    plt.show()

# 시퀀스별 변화량 시각화
plot_sequence_changes(average_changes, y)


### 변화량 80 ??

In [ ]:
import numpy as np

# 경로 설정
keypoints_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/filtered_keypoints_with_left_fixed.npy"
output_data_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/labeled_lstm_data_80threshold.npy"

# 파라미터 설정
SEQUENCE_LENGTH = 10  # 시퀀스 길이
THRESHOLD_FACTOR = 80  # 변화량이 80 이상인 경우 폭력으로 감지

# 데이터 준비 함수
def prepare_data_with_new_threshold(data):
    previous_coords = {}
    sequence = []
    labels = []

    # 프레임별 변화량 계산
    for frame, keypoints in sorted(data.items()):
        frame_changes = []
        for key in ["right_wrist", "right_elbow", "left_wrist", "left_elbow"]:
            current_coords = keypoints.get(key, None)
            if key in previous_coords and current_coords:
                # 변화량 계산
                delta = abs(current_coords[0] - previous_coords[key][0]) + abs(current_coords[1] - previous_coords[key][1])
                frame_changes.append(delta)
            else:
                frame_changes.append(0)  # 키포인트가 없으면 변화량 0으로 설정
            if current_coords:
                previous_coords[key] = current_coords  # 이전 프레임 업데이트
        sequence.append(frame_changes)

    # 시퀀스 생성 및 라벨링
    X, y = [], []
    for i in range(len(sequence) - SEQUENCE_LENGTH):
        sub_sequence = sequence[i:i+SEQUENCE_LENGTH]  # 시퀀스 길이만큼 추출
        high_change = any(max(frame) >= THRESHOLD_FACTOR for frame in sub_sequence)  # 80 이상 변화량 존재 여부
        label = 1 if high_change else 0
        X.append(sub_sequence)
        y.append(label)

    return np.array(X), np.array(y)

# 키포인트 데이터 불러오기
print("Loading keypoints data...")
keypoints_data = np.load(keypoints_file, allow_pickle=True).item()

# 변화량 데이터 준비
print("Calculating coordinate changes with new threshold...")
X, y = prepare_data_with_new_threshold(keypoints_data)

# 결과 저장
print(f"Saving relabeled data to {output_data_file}...")
np.save(output_data_file, {"X": X, "y": y})

# 데이터 확인
print("Data preparation complete.")
print(f"Shape of X (input sequences): {X.shape}")
print(f"Shape of y (labels): {y.shape}")
print(f"Number of Violent (Label=1): {np.sum(y)}")
print(f"Number of Non-Violent (Label=0): {len(y) - np.sum(y)}")


In [46]:
### 80 시각화

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 경로 설정
data_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/labeled_lstm_data_80threshold.npy"

# 데이터 불러오기
data = np.load(data_file, allow_pickle=True).item()
X, y = data["X"], data["y"]

# 데이터 정보 출력
print("Shape of X (input sequences):", X.shape)  # (시퀀스 수, 시퀀스 길이, 키포인트 수)
print("Shape of y (labels):", y.shape)          # (시퀀스 수,)
print(f"Number of Violent (Label=1): {np.sum(y)}")
print(f"Number of Non-Violent (Label=0): {len(y) - np.sum(y)}")

# 시퀀스별 평균 변화량 계산
average_changes = X.mean(axis=(1, 2))  # 시퀀스 단위로 평균 변화량 계산

# 시각화 함수
def plot_labeled_sequences(average_changes, labels):
    plt.figure(figsize=(12, 6))
    
    for i, (change, label) in enumerate(zip(average_changes, labels)):
        color = 'red' if label == 1 else 'blue'  # 폭력(1)은 빨간색, 비폭력(0)은 파란색
        plt.scatter(i, change, color=color, s=10)

    plt.title("Average Coordinate Changes for Each Sequence (New Threshold)")
    plt.xlabel("Sequence Index")
    plt.ylabel("Average Coordinate Changes")
    plt.grid(True)
    plt.legend(["Violent (Label=1)", "Non-Violent (Label=0)"], loc="upper right")
    plt.show()

# 그래프 시각화
plot_labeled_sequences(average_changes, y)


In [49]:
### 80이상.....

In [ ]:
import numpy as np

# 경로 설정
keypoints_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/filtered_keypoints_with_left_fixed.npy"
output_data_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/labeled_lstm_data_avg80.npy"

# 파라미터 설정
SEQUENCE_LENGTH = 10  # 시퀀스 길이
THRESHOLD_FACTOR = 80  # 평균 변화량이 80 이상인 경우 폭력으로 라벨링

# 데이터 준비 함수
def prepare_data_with_avg_threshold(data):
    previous_coords = {}
    sequence = []

    # 프레임별 변화량 계산
    for frame, keypoints in sorted(data.items()):
        frame_changes = []
        for key in ["right_wrist", "right_elbow", "left_wrist", "left_elbow"]:
            current_coords = keypoints.get(key, None)
            if key in previous_coords and current_coords:
                # 변화량 계산
                delta = abs(current_coords[0] - previous_coords[key][0]) + abs(current_coords[1] - previous_coords[key][1])
                frame_changes.append(delta)
            else:
                frame_changes.append(0)  # 키포인트가 없으면 변화량 0
            if current_coords:
                previous_coords[key] = current_coords
        sequence.append(frame_changes)

    # 시퀀스 생성 및 라벨링
    X, y = [], []
    for i in range(len(sequence) - SEQUENCE_LENGTH):
        sub_sequence = sequence[i:i+SEQUENCE_LENGTH]
        avg_change = np.mean(sub_sequence)  # 시퀀스 평균 변화량 계산
        label = 1 if avg_change >= THRESHOLD_FACTOR else 0  # 평균 변화량이 80 이상인 경우 라벨 1
        X.append(sub_sequence)
        y.append(label)

    return np.array(X), np.array(y)

# 키포인트 데이터 불러오기
print("Loading keypoints data...")
keypoints_data = np.load(keypoints_file, allow_pickle=True).item()

# 변화량 데이터 준비
print("Calculating coordinate changes with average threshold...")
X, y = prepare_data_with_avg_threshold(keypoints_data)

# 결과 저장
print(f"Saving relabeled data to {output_data_file}...")
np.save(output_data_file, {"X": X, "y": y})

# 데이터 확인
print("Data preparation complete.")
print(f"Shape of X (input sequences): {X.shape}")
print(f"Shape of y (labels): {y.shape}")
print(f"Number of Violent (Label=1): {np.sum(y)}")
print(f"Number of Non-Violent (Label=0): {len(y) - np.sum(y)}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 경로 설정
data_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/labeled_lstm_data_avg80.npy"

# 데이터 불러오기
data = np.load(data_file, allow_pickle=True).item()
X, y = data["X"], data["y"]

# 데이터 정보 출력
print("Shape of X (input sequences):", X.shape)  # (시퀀스 수, 시퀀스 길이, 키포인트 수)
print("Shape of y (labels):", y.shape)          # (시퀀스 수,)
print(f"Number of Violent (Label=1): {np.sum(y)}")
print(f"Number of Non-Violent (Label=0): {len(y) - np.sum(y)}")

# 시퀀스별 평균 변화량 계산
average_changes = X.mean(axis=(1, 2))  # 시퀀스 단위로 평균 변화량 계산

# 시각화 함수
def plot_labeled_sequences(average_changes, labels):
    plt.figure(figsize=(12, 6))
    
    for i, (change, label) in enumerate(zip(average_changes, labels)):
        color = 'red' if label == 1 else 'blue'  # 폭력(1)은 빨간색, 비폭력(0)은 파란색
        plt.scatter(i, change, color=color, s=10)

    plt.title("Average Coordinate Changes for Each Sequence (Updated Threshold)")
    plt.xlabel("Sequence Index")
    plt.ylabel("Average Coordinate Changes")
    plt.grid(True)
    plt.legend(["Violent (Label=1)", "Non-Violent (Label=0)"], loc="upper right")
    plt.show()

# 그래프 시각화
plot_labeled_sequences(average_changes, y)


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim

# 데이터 로드
data_file = "/home/zeus/ws/AEGISVISION/src/jhlstm/data/viol_1217/labeled_lstm_data_avg80.npy"
data = np.load(data_file, allow_pickle=True).item()
X, y = data["X"], data["y"]

# 데이터 전처리: 학습/테스트 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_test = torch.tensor(X_train, dtype=torch.float32), torch.tensor(X_test, dtype=torch.float32)
y_train, y_test = torch.tensor(y_train, dtype=torch.long), torch.tensor(y_test, dtype=torch.long)

# LSTM 모델 정의
class LSTMModel(nn.Module):
    def __init__(self, input_size=4, hidden_size=64, num_layers=2, output_size=2):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        _, (hn, _) = self.lstm(x)
        out = self.fc(hn[-1])  # 마지막 LSTM 레이어의 hidden state 사용
        return out

# 모델 초기화
model = LSTMModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 모델 학습
epochs = 30
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

# 모델 평가
model.eval()
with torch.no_grad():
    y_pred = model(X_test)
    y_pred_labels = torch.argmax(y_pred, dim=1)
    accuracy = (y_pred_labels == y_test).sum().item() / len(y_test)
    print(f"Test Accuracy: {accuracy:.4f}")


### 평가 !!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# 혼동 행렬 생성
conf_matrix = confusion_matrix(y_test, y_pred_labels)

# 혼동 행렬 시각화
plt.figure(figsize=(6, 5))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap="Blues", xticklabels=["Non-Violent", "Violent"], yticklabels=["Non-Violent", "Violent"])
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

# 정밀도, 재현율, F1-Score 출력
print("Classification Report:")
print(classification_report(y_test, y_pred_labels, target_names=["Non-Violent", "Violent"]))


In [ ]:
# 모델 저장
torch.save(model.state_dict(), "violence_detection_lstm.pth")
print("Model saved successfully!")


In [ ]:
import os
print("Current working directory:", os.getcwd())
